## Elo Merchant Category Recommendation
## 1. Descriptive Statistics 2.
End date: _2019. february 19._<br/>

This tutorial notebook is the second part of a seriers for [Elo Mechant Category Recommendation](https://www.kaggle.com/c/elo-merchant-category-recommendation) contest organized by Elo, one of the largest payment brands in Brazil. It has built partnerships with merchants in order to offer promotions or discounts to cardholders. The objective of the competition is to identify and serve the most relevant opportunities to individuals, by uncovering signals in customer loyalty. The input files are available from the [download](https://www.kaggle.com/c/elo-merchant-category-recommendation/data) section of the contest:

- **train.csv**,  **test.csv**: list of `card_ids` that can be used for training and testing
- **historical_transactions.csv**: contains up to 3 months' worth of transactions for every card at any of the provided `merchant_ids`
- **new_merchant_transactions.csv**: contains the transactions at new merchants (`merchant_ids` that this particular `card_id` 
has not yet visited) over a period of two months
- **merchants.csv**: contains aggregate information for each `merchant_id` represented in the data set

Contents of this notebook:
* [Transactions](#Transaction)
* [Train and Test data](#Train-and-Test-data)
* [Merchants](#Merchants)
* [Merged](#Merged)

In [1]:
import os
import gc
import math
import random
import warnings
import datetime
import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

random.seed(1)
threshold = 0.5

In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Starting memory usage: {:5.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min >= np.iinfo(np.int64).min and c_max <= np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min >= np.finfo(np.float16).min and c_max <= np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min >= np.finfo(np.float32).min and c_max <= np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Reduced memory usage: {:5.2f} MB ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

## Transactions

In [3]:
df_hist_trans = pd.read_csv("input/historical_transactions.csv")
print("{:,} records and {} features in historical transactions set.".format(df_hist_trans.shape[0], df_hist_trans.shape[1]))

df_hist_trans = reduce_mem_usage(df_hist_trans)

29,112,361 records and 14 features in historical transactions set.
Starting memory usage: 3109.54 MB
Reduced memory usage: 1749.11 MB (43.7% reduction)


In [4]:
df_hist_trans[:3]

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37


In [9]:
df_new_trans = pd.read_csv("input/new_merchant_transactions.csv")
print("{:,} records and {} features in new transactions set.".format(df_new_trans.shape[0], df_new_trans.shape[1]))

df_new_trans = reduce_mem_usage(df_new_trans)

1,963,031 records and 14 features in new transactions set.
Starting memory usage: 209.67 MB
Reduced memory usage: 114.20 MB (45.5% reduction)


In [10]:
df_new_trans[:3]

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557617,2018-03-11 14:57:36,1.0,9,19
1,Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569336,2018-03-19 18:53:37,1.0,9,19
2,Y,C_ID_415bb3a509,330,N,1,B,507,M_ID_ad5237ef6b,2,-0.551270,2018-04-26 14:08:44,1.0,9,14


Mapping categorical variables to integers

In [11]:
df_hist_trans['authorized_flag'] = df_hist_trans['authorized_flag'].map({'N': 0, 'Y': 1})
df_new_trans['authorized_flag'] = df_new_trans['authorized_flag'].map({'N': 0, 'Y': 1})

df_hist_trans['category_1'] = df_hist_trans['category_1'].map({'N': 0, 'Y': 1})
df_new_trans['category_1'] = df_new_trans['category_1'].map({'N': 0, 'Y': 1})

df_hist_trans['category_2'] = pd.to_numeric(df_hist_trans['category_2'])
df_new_trans['category_2'] = pd.to_numeric(df_new_trans['category_2'])

df_hist_trans['category_3'] = df_hist_trans['category_3'].map({'A': 0, 'B': 1, 'C': 2})
df_new_trans['category_3'] = df_new_trans['category_3'].map({'A': 0, 'B': 1, 'C': 2})

In [12]:
df_new_trans[:3]

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,1,C_ID_415bb3a509,107,0,1,1.0,307,M_ID_b0c793002c,1,-0.557617,2018-03-11 14:57:36,1.0,9,19
1,1,C_ID_415bb3a509,140,0,1,1.0,307,M_ID_88920c89e8,1,-0.569336,2018-03-19 18:53:37,1.0,9,19
2,1,C_ID_415bb3a509,330,0,1,1.0,507,M_ID_ad5237ef6b,2,-0.551270,2018-04-26 14:08:44,1.0,9,14


Extracting features from date

In [13]:
def create_date_features(df, source_column, preposition):
    df[preposition + '_year'] = df[source_column].dt.year
    df[preposition + '_month'] = df[source_column].dt.month
    df[preposition + '_day'] = df[source_column].dt.day
    df[preposition + '_hour'] = df[source_column].dt.hour
    df[preposition + '_weekofyear'] = df[source_column].dt.weekofyear
    df[preposition + '_dayofweek'] = df[source_column].dt.dayofweek
    df[preposition + '_quarter'] = df[source_column].dt.quarter
    
    return df

In [14]:
df_new_trans['purchase_date'] = pd.to_datetime(df_new_trans['purchase_date'])
df_new_trans = create_date_features(df_new_trans, 'purchase_date', 'purchase')

In [15]:
df_hist_trans['purchase_date'] = pd.to_datetime(df_hist_trans['purchase_date'])
df_hist_trans = create_date_features(df_hist_trans, 'purchase_date', 'purchase')

In [16]:
df_hist_trans[:3]

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,...,category_2,state_id,subsector_id,purchase_year,purchase_month,purchase_day,purchase_hour,purchase_weekofyear,purchase_dayofweek,purchase_quarter
0,1,C_ID_4e6213e9bc,88,0,0,0.0,80,M_ID_e020e9b302,-8,-0.703331,...,1.0,16,37,2017,6,25,15,25,6,2
1,1,C_ID_4e6213e9bc,88,0,0,0.0,367,M_ID_86ec983688,-7,-0.733128,...,1.0,16,16,2017,7,15,12,28,5,3
2,1,C_ID_4e6213e9bc,88,0,0,0.0,80,M_ID_979ed661fc,-6,-0.720386,...,1.0,16,37,2017,8,9,22,32,2,3


Extra aggregator functions

In [17]:
def mode(series):
    ''' Return the mode of the series '''
    if len(series.mode()) > 0:
        return series.mode().iloc[0]
    else:
        return np.nan

def nancnt(series):
    ''' Returns the count of NaN values '''
    return series.isnull().sum()

def nanperc(series):
    ''' Returns the percentile of NaN values '''
    return 100 * series.isnull().sum() / len(series)

In [20]:
def aggregate_transactions(df, prefix):  
    agg_funcs = {
        'authorized_flag': ['sum', 'mean'],
        'category_1': ['sum', 'mean'],
        'category_2': ['sum', 'mean', mode, nancnt, nanperc],
        'category_3': ['sum', 'mean', mode, nancnt, nanperc],
        
        'city_id': ['nunique', mode, nancnt, nanperc],
        
        'installments': ['sum', 'median', 'mean', 'max', 'min', 'std', mode, nancnt, nanperc],

        'merchant_id': ['nunique', mode, nancnt, nanperc],
        'merchant_category_id': ['nunique', mode, nancnt, nanperc],
        'month_lag': ['min', 'max', 'mean'],

        'state_id': ['nunique', mode, nancnt, nanperc],
        'subsector_id': ['nunique', mode, nancnt, nanperc],
        
        'purchase_amount': ['sum', 'mean', 'max', 'min', 'std', mode],
        'purchase_year': ['mean', 'median', 'max', 'min', 'std', mode],
        'purchase_month': ['mean', 'median', 'max', 'min', 'std', mode],
        'purchase_day': ['mean', 'median', 'max', 'min', 'std', mode],
        'purchase_hour': ['mean', 'median', 'max', 'min', 'std', mode],
        'purchase_weekofyear': ['mean', 'median', 'max', 'min', 'std', mode],
        'purchase_dayofweek': ['mean', 'median', 'max', 'min', 'std', mode],
        'purchase_quarter': ['mean', 'median', 'max', 'min', 'std', mode]
    }
    df_agg = df.groupby('card_id').agg(agg_funcs)
    df_agg.columns = [prefix + '_'.join(col).strip() for col in df_agg.columns.values]
    df_agg.reset_index(inplace=True)

    df = (df_agg.groupby('card_id').size().reset_index(name='{}transactions_count'.format(prefix)))
    df_agg = pd.merge(df, df_agg, on='card_id', how='left')

    return df_agg

In [21]:
df_new_trans = aggregate_transactions(df_new_trans, prefix='new_')

In [22]:
df_new_trans[:3]

,card_id,new_transactions_count,new_authorized_flag_sum,new_authorized_flag_mean,new_category_1_sum,new_category_1_mean,new_category_2_sum,new_category_2_mean,new_category_2_mode,new_category_2_nancnt,...,new_purchase_dayofweek_max,new_purchase_dayofweek_min,new_purchase_dayofweek_std,new_purchase_dayofweek_mode,new_purchase_quarter_mean,new_purchase_quarter_median,new_purchase_quarter_max,new_purchase_quarter_min,new_purchase_quarter_std,new_purchase_quarter_mode
0,C_ID_00007093c1,1,2,1,0,0.000000,4.0,2.000000,1.0,0.0,...,1,0,0.707107,0,2.000000,2.0,2,2,0.000000,2
1,C_ID_0001238066,1,26,1,2,0.076923,35.0,1.521484,1.0,3.0,...,6,0,1.799573,4,1.346154,1.0,2,1,0.485165,1
2,C_ID_0001506ef0,1,2,1,0,0.000000,6.0,3.000000,3.0,0.0,...,4,3,0.707107,3,1.000000,1.0,1,1,0.000000,1


In [23]:
df_new_trans.to_csv('input/trans_new_agg.csv')

In [24]:
df_hist_trans = aggregate_transactions(df_hist_trans, prefix='hist_')

MemoryError: 

In [ ]:
df_hist_trans[:3]

In [ ]:
df_hist_trans.to_csv('input/trans_hist_agg.csv')

## Train and test data

In [ ]:
df_train = pd.read_csv("input/train.csv")
df_test = pd.read_csv("input/test.csv")
print("{:,} records and {} features in train set.".format(df_train.shape[0], df_train.shape[1]))
print("{:,} records and {} features in test set.".format(df_test.shape[0], df_test.shape[1]))

df_train = reduce_mem_usage(df_train)
df_test = reduce_mem_usage(df_test)

In [ ]:
df_train[:3]

In [ ]:
df_test[:3]

## Merchants

In [ ]:
df_merch = pd.read_csv("input/merchants.csv")
print("{:,} records and {} features in merchant set.".format(df_merch.shape[0], df_merch.shape[1]))

df_merch = reduce_mem_usage(df_merch)

In [ ]:
df_merch[:3]

## Merged

Join the data of the merchants and the transactions to the training and test set.

In [ ]:
df_train_p = pd.merge(df_train_p, df_new_trans_p_agg, on='card_id', how='left')
df_test_p = pd.merge(df_test_p, df_new_trans_p_agg, on='card_id', how='left')
del df_new_trans_p_agg
gc.collect()

In [ ]:
df_train_p = pd.merge(df_train_p, df_hist_trans_p_agg, on='card_id', how='left')
df_test_p = pd.merge(df_test_p, df_hist_trans_p_agg, on='card_id', how='left')
del df_hist_trans_p_agg
gc.collect()

In [ ]:
data = pd.merge(df_merch, df_new_trans, on = 'merchant_id', suffixes=['_merch', '_trans'])

In [ ]:
data = pd.merge(df_merch, df_hist_trans, on = 'merchant_id', suffixes=['_merch', '_trans'])

In [ ]:
data.columns

In [ ]:
data[['card_id',
      'merchant_category_id_merch', 'merchant_category_id_trans',
      'subsector_id_merch', 'subsector_id_trans',
      'category_1_merch', 'category_1_trans',
      'category_2_merch', 'category_2_trans',
      'city_id_merch', 'city_id_trans',
      'state_id_merch', 'state_id_trans']][:10]

df_merch[df_merch['active_months_lag12'] < 20]['active_months_lag12'].hist(bins=12)

In [ ]:
n_mc = len(data[data['merchant_category_id_merch'] != data['merchant_category_id_trans']])
print('There are {:,} records, where the merchant_id\'s don\'t match, it is {:.2f}% of the total number of records.'.format(
    n_mc,
    100*n_mc/data.shape[0]
))

n_ss = len(data[data['subsector_id_merch'] != data['subsector_id_trans']])
print('There are {:,} records, where the subsector_id\'s don\'t match, it is {:.2f}% of the total number of records.'.format(
    n_ss,
    100*n_ss/data.shape[0]
))

n_c1 = len(data[data['category_1_merch'] != data['category_1_trans']])
print('There are {:,} records, where the category_1\'s don\'t match, it is {:.2f}% of the total number of records.'.format(
    n_c1,
    100*n_c1/data.shape[0]
))

n_c2 = len(data[data['category_2_merch'] != data['category_2_trans']])
print('There are {:,} records, where the category_2\'s don\'t match, it is {:.2f}% of the total number of records.'.format(
    n_c1,
    100*n_c1/data.shape[0]
))

n_c = len(data[data['city_id_merch'] != data['city_id_trans']])
print('There are {:,} records, where the city_id\'s don\'t match, it is {:.2f}% of the total number of records.'.format(
    n_c,
    100*n_c/data.shape[0]
))

n_s = len(data[data['state_id_merch'] != data['state_id_trans']])
print('There are {:,} records, where the state_id\'s don\'t match, it is {:.2f}% of the total number of records.'.format(
    n_s,
    100*n_s/data.shape[0]
))

In [ ]:
#data = pd.merge(data, df_train, on = 'card_id')

Split the purchase date to date and time.

In [ ]:
df_new_trans['purchase_time'] = df_new_trans['purchase_date'].str.split(' ')
df_new_trans['purchase_date'] = df_new_trans['purchase_time'].str[0]
df_new_trans['purchase_time'] = df_new_trans['purchase_time'].str[1]

Then extract the day of the week from the day of the date. The `pd.Categorical` function sets the order of the days of the week.

In [ ]:
def get_weekday(date_string):
    date = datetime.datetime.strptime(date_string, '%Y-%m-%d')
    return calendar.day_name[date.weekday()]

df_new_trans['purchase_weekday'] = df_new_trans['purchase_date'].apply(lambda x: get_weekday(x))

day_labels = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df_new_trans['purchase_weekday'] = pd.Categorical(df_new_trans['purchase_weekday'], categories = day_labels, ordered = True)

The plot above shows the target changes over the week.

The default behavior in Seaborn is to aggregate the multiple measurements at each x value by plotting the mean and the 95% confidence interval around the mean. (More info on [Aggregation and representing uncertainty](https://seaborn.pydata.org/tutorial/relational.html#aggregation-and-representing-uncertainty))

In [ ]:
ax = sns.lineplot(x = "purchase_weekday", y = "target", markers = False, dashes = True, data = df_new_trans)
plt.xticks(rotation = 45)
ax.set_title('Target Variable Changes over Purchase Week')
ax.set_xlabel('Purchase Weekday')

In [ ]:
def get_month(date_string, kind = 'month'):
    if kind == 'month':
        date = datetime.datetime.strptime(date_string, '%Y-%m')
    elif kind == 'day':
        date = datetime.datetime.strptime(date_string, '%Y-%m-%d')
    return date.strftime("%B")

data['purchase_month'] = data['purchase_date'].apply(lambda x: get_month(x, kind='day'))
data['first_active_month2'] = data['first_active_month'].apply(lambda x: get_month(x))

month_labels = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
data['purchase_month'] = pd.Categorical(data['purchase_month'], categories = month_labels, ordered = True)
data['first_active_month2'] = pd.Categorical(data['first_active_month2'], categories = month_labels, ordered = True)

In [ ]:
ax = sns.lineplot(x = "purchase_month", y = "target", markers = True, dashes = False, data = data)
plt.xticks(rotation = 45)
ax.set_title('Target Variable Changes over Purchase Month')
ax.set_xlabel('Purchase Month')

The plot shows the target changes over the month.

In [ ]:
data['first_active_year'] = data['first_active_month'].str[:4]

year_labels = ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']
data['first_active_year'] = pd.Categorical(data['first_active_year'], categories = year_labels, ordered = True)

In [ ]:
ax = sns.lineplot(x = "first_active_year", y = "target", markers = True, dashes = False, data = data)
plt.xticks(rotation = 45)
ax.set_title('Target Variable Changes over the First Active Year')
ax.set_xlabel('First Active Year')

The plot shows the target changes over the first active year.

In [ ]:
ax = sns.lineplot(x = "first_active_month2", y = "target", markers = True, dashes = False, data = data)
plt.xticks(rotation = 45)
ax.set_title('Target Variable Changes over the First Active Month')
ax.set_xlabel('First Active Month')

The plot showst the target changes over the first active month.

In [ ]:
data['temp'] = data['purchase_time'].str.split(':')

def get_session(time_list):
    time_list[0] = int(time_list[0])
    if time_list[0] > 4 and time_list[0] < 12:
        return 'Morning'
    elif time_list[0] >= 12 and time_list[0] < 17:
        return 'Afternoon'
    elif time_list[0] >= 17 and time_list[0] < 21:
        return 'Evening'
    else:
        return 'Night'
    
data['purchase_session'] = data['temp'].apply(lambda x: get_session(x))

session_labels = ['Morning', 'Afternoon', 'Evening', 'Night']
data['purchase_session'] = pd.Categorical(data['purchase_session'], categories = session_labels, ordered = True)
data.drop('temp', axis = 1, inplace=True)

In [ ]:
ax = sns.lineplot(x = "purchase_session", y = "target", markers = True, dashes = False, data = data)
plt.xticks(rotation = 45)
ax.set_title('Target Variable Changes over Purchase Time of Day')
ax.set_xlabel('Purchase Time of Day')

The plot shows the target changes over the purchase time of day.

In [ ]:
ax = sns.catplot(x='purchase_weekday', y='target', hue='purchase_session', data=data, kind='bar', height=5, aspect=2)
ax.despine(left = True)
plt.xticks(rotation = 45)
ax.set_ylabels("target")
ax.set_xlabels('Weekday')

In [ ]:
def get_time_of_month_cat(date):
    date_temp = date.split('-')
    if int(date_temp[2]) < 10:
        time_of_month = 'Beginning'
    elif int(date_temp[2]) >= 10 and int(date_temp[2]) < 20:
        time_of_month = 'Middle'
    else:
        time_of_month = 'End'
    return time_of_month

data['time_of_month_cat'] = data['purchase_date'].apply(lambda x: get_time_of_month_cat(x))

tof_labels = ['Beginning', 'Middle', 'End']
data['time_of_month_cat'] = pd.Categorical(data['time_of_month_cat'], categories=tof_labels, ordered=True)

In [ ]:
ax = sns.catplot(x='purchase_month', y='target', hue='time_of_month_cat', data=data, kind='bar', height=5, aspect=2)
ax.despine(left = True)
plt.xticks(rotation = 45)
ax.set_ylabels("Target")
ax.set_xlabels('Purchase Time of Month')

The plot shows the target changes over the purchase time of month.

In [ ]:
def get_end_of_month(date):
    date_temp = date.split('-')
    if int(date_temp[2]) >= 25:
        end_of_month = 'Yes'
    else:
        end_of_month = 'No'
    return end_of_month

data['end_of_month'] = data['purchase_date'].apply(lambda x: get_end_of_month(x))

ax = sns.barplot(x = 'end_of_month', y = 'target', data = data)

* Két lépésben csinálni predikciót
    * Outlierek megkeresése (CatBoost, Autoencoder)
* A group_by csak card_id-re van, lehetne az üzletekre is aggregálni
* multikollinearitás: a felesleges változókat el kell dobni